In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.special import expit
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
from sklearn.metrics.pairwise import rbf_kernel,linear_kernel
from sklearn.metrics import mean_squared_error
from sklearn.utils.validation import check_X_y
from sklearn.svm import SVR
from sklearn.datasets import make_friedman1, make_friedman2, make_friedman3, load_boston
from sklearn.datasets import make_moons, make_circles, make_classification
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_gaussian_quantiles, make_circles
import time

In [ ]:
def IRLS(X, y, delta=1e-4, n_iter=20, tol=1e-3):
    n_samples, n_features = X.shape
    # y.shape (n_samples, n_features) n_feature=1?
    y=y.reshape((n_samples,))
    #print(y.shape)
    delta = delta * np.ones((y.shape))
    w_init = 1
    w = w_init * np.ones((y.shape)).reshape((-1,))
    W = np.diag(w)
    
    #print("W shape",W.shape)
    #print("w shape",w.shape)
    beta = np.dot(np.linalg.inv(np.dot(np.dot(X.T,W),X)),np.dot(np.dot(X.T,W),y))
    # beta shape 
    for i in range(n_iter):
        #w_=w
        print("iteration {}".format(i))
        beta_old=beta
        w = 1/(np.maximum(delta,np.absolute(y-np.dot(X,beta)))).reshape((-1,))
        W = np.diag(w)
        #print(w.shape)
        #print(W.shape)
        beta = np.dot(np.linalg.inv(np.dot(np.dot(X.T,W),X)),np.dot(np.dot(X.T,W),y))
        #print("beta shape",beta.shape)
        tolerance = np.sum(np.absolute(beta-beta_old))
        print("tolerance is {}".format(tolerance))
        if tolerance < tol and i >1:
            break
    return beta.reshape((-1,1))


In [ ]:
class RVR(BaseEstimator, RegressorMixin):
    def __init__(
        self,
        kernel='rbf',
        rbf_gamma=None,
        n_iter=5000,
        threshold_dif=1e-3,
        alpha=1e-6,
        beta=1.e-6,
        threshold_alpha=1e9,
        bias_used = True
    ):
        self.kernel = kernel
        self.rbf_gamma = rbf_gamma
        self.n_iter = n_iter
        self.threshold_dif = threshold_dif
        self.alpha = alpha
        self.beta = beta
        self.threshold_alpha = threshold_alpha
        self.bias_used = bias_used
        
    def get_params(self, deep=True):
        params = {
            'kernel': self.kernel,
            'rbf_gamma': self.rbf_gamma,
            'n_iter': self.n_iter,
            'threshold_dif': self.threshold_dif,
            'alpha': self.alpha,
            'beta': self.beta,
            'threshold_alpha': self.threshold_alpha,
            'bias_used': self.bias_used
        }
        return params
  
    def set_params(self, **params):
        for parameter, value in params.items():
            setattr(self, parameter, value)
        return self
    
    def apply_kernel(self, x, y):
        if x.shape[1]!=y.shape[1]:
            raise ValueError("Wrong shape of input")
        n_samples1, n_features=x.shape
        n_samples2, n_features=y.shape
        if self.kernel=='rbf':
            #print(x.shape)
            #print(y.shape)
            phi = rbf_kernel(x, y, self.rbf_gamma)
        
        elif self.kernel=='linear':
            phi = linear_kernel(x,y)
        elif self.kernel=='linear_spline':
            #print("x repeat shape",np.repeat(x,n_samples2,axis=1).shape)
            #print("y repeat shape",np.repeat(y,n_samples1,axis=1).shape)
            #a = np.repeat(x,3,axis=1)
            #b=np.repeat(y.T,3,axis=0)
            Min = np.minimum(np.repeat(x,n_samples2,axis=1),np.repeat(y.T,n_samples1,axis=0)) #( 100, 100)
            #print("x shape: {}".format(x.shape))
            #print(" shape: {}".format(y.shape))
            phi =1 + linear_kernel(x,y) + linear_kernel(x,y)*Min - (x+y.T)*np.square(Min)/2 + np.power(Min,3)/3
            #output= 1 + np.dot(x,y)+ np.dot(x,y)*min(x,y)- ((x + y)/2)*(min(x,y))**2 + ((min(x,y))**3)/3
            #print(np.amax(np.absolute (phi-output)))
        else:
            raise ValueError("rbf or linear kernel only!")
            
        if self.bias_used:
            phi = np.append(np.ones((phi.shape[0],1)), phi, axis=1)
        return phi  
        
    def prune(self):
        keep_alpha = self.alpha_ < self.threshold_alpha
        #if not np.any(self.alpha_[keep_alpha]):
        #    keep_alpha[0] = True
        #print("keep_alpha shape: {}".format(keep_alpha.shape))
        
        if self.bias_used:
            if not keep_alpha[0]:
                self.bias_used = False
            self.relevance_ = self.relevance_[keep_alpha[1:]]
        else:
             self.relevance_ = self.relevance_[keep_alpha]
        
        self.alpha_ = self.alpha_[keep_alpha]
        self.alpha_old = self.alpha_old[keep_alpha]
        #self.beta_ = self.beta_[keep_alpha]
        self.mu_ = self.mu_[keep_alpha]
        self.gamma_ = self.gamma_[keep_alpha]
        self.phi = self.phi[:,keep_alpha]
        #self.sigma_ = self.sigma_[:,keep_alpha]
        self.sigma_ = self.sigma_[np.ix_(keep_alpha,keep_alpha)]
        return self
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
        n_samples, n_features = X.shape
        self.relevance_ = X
        self.y = y
        
        self.phi = self.apply_kernel(X, X)
        
        n_basis_functions = self.phi.shape[1]  # if bias_used is True, n-basis_funtions = n_samples + 1 
        
       
        self.mu_ = np.zeros(n_basis_functions)
        self.alpha_ = self.alpha * np.ones(n_basis_functions)
        self.beta_ = self.beta
        #self.beta_ = np.ones(n_basis_functions) * (expit(np.dot(self.phi,self.w_mp_))/(1-expit(np.dot(self.phi,self.w_mp_))))
        #print(sel.beta_.shape)
        
        
        self.alpha_old = self.alpha_
        
        
        
        for i in range(self.n_iter):
            
            self.sigma_ = np.linalg.inv(self.beta_ * np.dot(self.phi.T, self.phi) + np.diag(self.alpha_))
            #print("sigma shape: {}".format(self.sigma_.shape))
            
            self.mu_ = self.beta_ * np.dot(np.dot(self.sigma_,self.phi.T),self.y)
            
            #self.sigma_ = np.linalg.inv(np.dot(np.dot(self.phi.T, np.diag(self.beta_)),self.phi) + np.diag(self.alpha_))
            #print("sigma shape: {}".format(self.sigma_.shape))
            
            #self.w_mp_ = (np.dot(np.dot(self.sigma_,self.phi.T),np.diag(self.beta_)),self.y)
            #print("mu shape: {}".format(self.mu_.shape))
            
            self.gamma_ = 1 - self.alpha_ * np.diag(self.sigma_)
            #print("gamma shape: {}".format(self.gamma_.shape))
            
            self.alpha_ = self.gamma_/(self.mu_ ** 2)
            #print("alpha shape: {}".format(self.alpha_.shape))
            
            self.beta_ = (n_samples - np.sum(self.gamma_))/(np.sum((np.absolute(self.y - np.dot(self.phi, self.mu_)))**2))
            #print("beta shape: {}".format(self.beta_.shape))
            
            self.prune()
            
            
            delta = np.amax(np.absolute(self.alpha_ - self.alpha_old))
            if delta < self.threshold_dif and i > 1:
                print("Iteration: {}".format(i))
                break
                
            self.alpha_old = self.alpha_
        return self
    def predict(self, X):
        #print(X.shape)
        #print(self.relevance_.shape)
        phi = self.apply_kernel(X, self.relevance_)
        #print(phi.shape)
        y = np.dot(phi, self.mu_)
        
#         if eval_MSE:
#             #print("sigma shape: {}".format(self.sigma_.shape))
#             #print("phi shape: {}".format(self.phi.shape))
            
#             MSE = (1/self.beta_) + np.dot(phi, np.dot(self.sigma_, phi.T))
#             return y, MSE[:, 0]
#         else:
#             return y
        return y
        
class RVC(BaseEstimator, ClassifierMixin):    
    def __init__(
        self,
        kernel='rbf',
        rbf_gamma=None,
        n_iter=5000,
        threshold_dif=1e-3,
        alpha=1e-6,
        beta=1.e-6,
        threshold_alpha=1e9,
        bias_used = False
        ):
        self.kernel = kernel
        self.rbf_gamma = rbf_gamma
        self.n_iter = n_iter
        self.threshold_dif = threshold_dif
        self.alpha = alpha
        self.beta = beta
        self.threshold_alpha = threshold_alpha
        self.bias_used = bias_used
        
    def get_params(self, deep=True):
        params = {
            'kernel': self.kernel,
            'rbf_gamma': self.rbf_gamma,
            'n_iter': self.n_iter,
            'threshold_dif': self.threshold_dif,
            'alpha': self.alpha,
            'beta': self.beta,
            'threshold_alpha': self.threshold_alpha,
            'bias_used': self.bias_used
        }
        return params
  
    def set_params(self, **params):
        for parameter, value in params.items():
            setattr(self, parameter, value)
        return self
    
    def apply_kernel(self, x, y):
        n_samples1, n_features=x.shape # (100,2)
        n_samples2, n_features=y.shape # (100,1)
        # 
        if self.kernel=='rbf':
            phi = rbf_kernel(x, y, self.rbf_gamma)
        
        elif self.kernel=='linear':
            phi = linear_kernel(x,y)
        elif self.kernel=='linear_spline':
            Min = np.minimum(np.repeat(x,n_samples2,axis=1),np.repeat(y.T,n_samples1,axis=0)) #( 100, 100)
            phi =1 + linear_kernel(x,y) + linear_kernel(x,y)*Min - (x+y.T)*np.square(Min)/2 + np.power(Min,3)/3
        else:
            raise ValueError("rbf or linear kernel only!")
           
        if self.bias_used:
            phi = np.append(np.ones((phi.shape[0],1)), phi, axis=1)
        #print("phi has the shape of {}".format(phi.shape))     
        return phi  
    
    def prune(self):
        keep_alpha = self.alpha_ < self.threshold_alpha
        #if not np.any(self.alpha_[keep_alpha]):
        #    keep_alpha[0] = True
        #print("keep_alpha shape: {}".format(keep_alpha.shape))
        
        if self.bias_used:
            if not keep_alpha[0]:
                self.bias_used = False
            self.relevance_ = self.relevance_[keep_alpha[1:]]
        else:
             self.relevance_ = self.relevance_[keep_alpha]
        
        self.alpha_ = self.alpha_[keep_alpha]
        self.alpha_old = self.alpha_old[keep_alpha]
        #self.beta_ = self.beta_[keep_alpha]
        self.w_mp_ = self.w_mp_[keep_alpha]
        self.gamma_ = self.gamma_[keep_alpha]
        self.phi = self.phi[:,keep_alpha]
        #print(self.phi.shape)
        #self.sigma_ = self.sigma_[:,keep_alpha]
        self.sigma_ = self.sigma_[np.ix_(keep_alpha,keep_alpha)]
        return self
    def fit(self, X, y):
        X, y = check_X_y(X, y)
        n_samples, n_features = X.shape # n_samples=100 n_features=2 (100,2)
        self.relevance_ = X
        self.y = y
        
        self.phi = self.apply_kernel(X, X)
        
        n_basis_functions = self.phi.shape[1]  # if bias_used is True, n-basis_funtions = n_samples + 1 
        
       
        
        self.alpha_ = self.alpha * np.ones(n_basis_functions)
        #self.beta_ = self.beta
        self.alpha_old = self.alpha_
        
        self.w_mp_ = np.zeros(n_basis_functions)
        #print("w_mp has the shape of",self.w_mp_.shape)
        #expit(np.dot(self.phi, self.w_mp_))
        #self.beta_ = np.ones(n_basis_functions)
        self.beta_ = np.ones(n_basis_functions) * (expit(np.dot(self.phi,self.w_mp_))/(1-expit(np.dot(self.phi,self.w_mp_))))
        for i in range(self.n_iter):
            #print(self.phi.shape)
            self.sigma_ = np.linalg.inv(np.dot(np.dot(self.phi.T, np.diag(self.beta_)),self.phi) + np.diag(self.alpha_))
            #self.sigma_ = np.linalg.inv(self.beta_ * np.dot(self.phi.T, self.phi) + np.diag(self.alpha_))
            #print("sigma shape: {}".format(self.sigma_.shape))
            self.w_mp_ = np.dot(np.dot(np.dot(self.sigma_,self.phi.T),np.diag(self.beta_)),self.y)
            #self.mu_ = self.beta_ * np.dot(np.dot(self.sigma_,self.phi.T),self.y)
            #print("mu shape: {}".format(self.mu_.shape))
            
            self.gamma_ = 1 - self.alpha_ * np.diag(self.sigma_)
            #print("gamma shape: {}".format(self.gamma_.shape))
            #print("w_mp has the shape of {}".format(self.w_mp_.shape))
            #tmp=np.square(self.w_mp_)
            #print("w_mp has the shape of: {}".format(tmp.shape))
            self.alpha_ = self.gamma_/np.square(self.w_mp_)
            #print("alpha shape: {}".format(self.alpha_.shape))
            
            self.beta_ = (expit(np.dot(self.phi,self.w_mp_))/(1-expit(np.dot(self.phi,self.w_mp_))))
            #self.beta_ = (n_samples - np.sum(self.gamma_))/(np.sum((np.absolute(self.y - np.dot(self.phi, self.w_mp_)))**2))
            #print("beta shape: {}".format(self.beta_.shape))
            
            self.prune()
            
            #delta = np.amax(np.absolute(self.beta_ - self.beta_old))
            delta = np.amax(np.absolute(self.alpha_ - self.alpha_old))
            if delta < self.threshold_dif and i > 1:
                print("Iteration: {}".format(i))
                break
                
            self.alpha_old = self.alpha_
        return self
    def predict(self, X):
        phi = self.apply_kernel(X, self.relevance_)
        #print(phi.shape)
        y = np.dot(phi, self.w_mp_)
        
        #print("y before sigmoid: ",y)
        y=expit(y)
        return y
        #return np.column_stack((1-y, y))

In [ ]:
def generate_data(N, sigma):
    #X = np.sort(20 * np.random.rand(N, 1)-10, axis=0)
    X=np.linspace(-10,10,N).reshape((N,1))
   
    y = np.sin(X)/X    # rbf kernel
    #y = 2*X + 10       # linear kernel
    y=y.ravel()
    noise = np.random.normal(0, sigma, N)
    #noise = 0
    t = y + noise
    #y[::5] += sigma * (0.5 - np.random.rand(int(N/5)))
    #t=y+np.random.normal(0,0.2)
    return X, t


# N=100    # 
# The running time for RVM is 0.10185384750366211
# The length of relevance vetor is : 9
# The running time for SVM is 0.00142669677734375
# The length of support vetor is : 62


# N=200    
# The running time for RVM is 0.8798322677612305
# The length of relevance vetor is : 9
# The running time for SVM is 0.0034551620483398438
# The length of support vetor is : 122

# N=300
# The running time for RVM is 0.6392791271209717
# The length of relevance vetor is : 9
# The running time for SVM is 0.006904125213623047
# The length of support vetor is : 190

# N=400   
# The running time for RVM is 1.4351470470428467
# The length of relevance vetor is : 9
# The running time for SVM is 0.012686967849731445
# The length of support vetor is : 256

# N=500
# The running time for RVM is 0.8161935806274414
# The length of relevance vetor is : 9
# The running time for SVM is 0.015553712844848633
# The length of support vetor is : 295

#N=600
#N=700
#N=800      # 2.68s
#N=900
# N=1000      # 4.22

X,y = generate_data(N, 0.2)
print(y.shape)

In [ ]:
Train = np.loadtxt('synth.tr.txt')

X=np.zeros((Train.shape[0],2))
y=np.zeros((Train.shape[0],1))

X[:,0]=Train[:,0]
X[:,1]=Train[:,1]
y=Train[:,2]
random.seed(5)
idx = np.random.choice(250,100)
X = X[idx]
y = y[idx]


In [ ]:
#X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
#                           random_state=1, n_clusters_per_class=1)
#X, Y = make_gaussian_quantiles(mean =[-2,2], cov=[1],n_features=2, n_classes=2, random_state=1)
#X,y= make_circles(noise=0.02, factor=0.8, random_state=1)
print("X shape is",X.shape)
print("y shape is",y.shape)
#rng = np.random.RandomState(2)
#X += 2 * rng.uniform(size=X.shape)
#linearly_separable = (X, y)
#print(X)
rvc_rbf=RVC(kernel='rbf',rbf_gamma=10,threshold_alpha=1e7)
y_rvc = rvc_rbf.fit(X, y).predict(X)
print("y_rvc",y_rvc)
print("y_rvc",y_rvc[:, 1])
print("y_rvc has the shape of",y_rvc.shape)
res = np.empty(y.shape[0])
res[y_rvc[:, 1] <= 0.5] = 0
res[y_rvc[:, 1] >= 0.5] = 1
print(res)
#print("deviation is",1/np.sqrt(rvc_rbf.beta_))
MSE_rvc = mean_squared_error(y, res)
#MSE_rvc=np.sum(np.abs(y-y_rvc))
#print(np.abs(y-y_rvc))
print("MSE is",MSE_rvc)
print("The  length of relevance vector is:",rvc_rbf.relevance_.shape[0])
plt.figure()
cm_bright = ListedColormap(['#FF0000', '#0000FF'])
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cm_bright,edgecolors='k')
plt.figure()
#plt.scatter(X[:, 0], X[:, 1], c=y_rvc,cmap=cm_bright,edgecolors='k')
plt.scatter(X[:, 0], X[:, 1], c=res,edgecolors='k')
plt.scatter(rvc_rbf.relevance_[:, 0], rvc_rbf.relevance_[:, 1],marker="s",c='y', edgecolors='k')
#print("y orignal is",y)
#print("y predict is ",y_rvc)

In [ ]:
# rbf kernel
X_test,y_test = generate_data(N, 0.2)
start = time.time()
#rvr_lin_spline = RVR(kernel='linear_spline')
rvr_rbf = RVR(kernel='rbf',threshold_alpha=1e10,rbf_gamma=0.05)
#svr_rbf = SVR(kernel='rbf')
y_rvr = rvr_rbf.fit(X,y).predict(X_test)
#y_svr = svr_rbf.fit(X,y).predict(X)
#y_rvr_lin_spline=rvr_lin_spline.fit(X,y).predict(X_test)
end = time.time()
MSE_rvr = mean_squared_error(y_test, y_rvr)
print('MSE for RVM is',MSE_rvr)
print('The running time for RVM is',end-start)
print("The length of relevance vetor is : {}".format(len(rvr_lin_spline.relevance_)))
# linear kernel


start = time.time()
svr_rbf = SVR(kernel='rbf',gamma=0.05)
#y_rvr = rvr_rbf.fit(X,y).predict(X)
y_svr = svr_rbf.fit(X,y).predict(X_test)
end = time.time()
print('The running time for SVM is',end-start)
print("The length of support vetor is : {}".format(len(svr_rbf.support_vectors_)))


#vr_lin = RVR(kernel='linear')
#y_rvr = rvr_lin.fit(X, y).predict(X)
#svr_lin = SVR(kernel='linear')
#y_svr = svr_lin.fit(X, y).predict(X)

# linear kernel


# print("estimated sigma is : {}".format(1/np.sqrt(rvr_rbf.beta_)))
# print("The length of relevance vetor is : {}".format(len(rvr_rbf.relevance_)))
# plt.figure()
# lw=2
# plt.scatter(X, y, color='darkorange', label='data')
# plt.plot(X,np.sin(X)/X,'b--',color='r')
# #plt.plot(X, y_rvr, 'r-',color='navy', label='RBF Model')
# #plt.plot(X, y_svr, 'r-',color='navy', lw=lw, label='Linear Spline Model')
# plt.plot(X, y_rvr_lin_spline, 'r-',color='navy', lw=lw, label='Linear Spline Model')
# #plt.scatter(rvr_rbf.relevance_, np.sin(rvr_rbf.relevance_)/rvr_rbf.relevance_, color='navy', label='Relevance vector')
# #plt.scatter(rvr_lin_spline.relevance_, np.sin(rvr_lin_spline.relevance_)/rvr_lin_spline.relevance_, color='navy', label='Relevance vector')
# plt.xlabel('data')
# plt.ylabel('target')
# plt.title('Relevance Vector Regression')
# #plt.legend()
# plt.figure()
# plt.plot(X,np.sin(X)/X,'b--',color='r')
# plt.scatter(X, y, color='darkorange', label='data')
# plt.plot(X, y_rvr, 'r-',color='navy', label='RBF Model')

# plt.legend()
# plt.show()

# print("The length of support vetor is : {}".format(len(svr_lin.support_vectors_)))
# plt.figure()
# lw=2
# #plt.scatter(X, y, color='darkorange', label='data')
# plt.plot(X, y_svr, color='navy', lw=lw, label='RBF model')
# plt.scatter(svr_lin.support_vectors_, 2 * svr_lin.support_vectors_ + 10,color='navy', label='Support vector')
# plt.xlabel('data')
# plt.ylabel('target')
# plt.title('Support Vector Regression')
# plt.legend()
# plt.show()

In [64]:
# load dataset 
#X, y = make_friedman1()
#X, y = make_friedman2()
#X, y = make_friedman3()
X, y = load_boston(return_X_y=True)

# SVR and RVR fit
rvr_lin = RVR(kernel='linear',threshold_alpha=1e10) # for friedman3, Boston
y_rvr = rvr_lin.fit(X, y).predict(X)
svr_lin = SVR(kernel='linear')
y_svr = svr_lin.fit(X, y).predict(X)
MSE_rvr = np.sqrt(mean_squared_error(y, y_rvr))
MSE_svr = np.sqrt(mean_squared_error(y, y_svr))

print("The length of relevance vetor is : {}".format(len(svr_lin.support_vectors_)))
print("The length of relevance vetor is : {}".format(len(rvr_lin.relevance_)))
print("RMS for SVM is : {}".format(MSE_svr))
print("RMS for RVM is : {}".format(MSE_rvr))



The length of relevance vetor is : 488
The length of relevance vetor is : 9
RMS for SVM is : 5.003661865062167
RMS for RVM is : 4.784697682102961


In [ ]:
# rbf kernel

print("estimated sigma is : {}".format(1/np.sqrt(rvr_rbf.beta_)))
print("The length of relevance vetor is : {}".format(len(rvr_rbf.relevance_)))
plt.figure()
lw=2
plt.scatter(X, y, color='darkorange', label='data')
plt.plot(X, y_rvr, color='navy', lw=lw, label='RBF model')
plt.scatter(rvr_rbf.relevance_, np.sin(rvr_rbf.relevance_)/rvr_rbf.relevance_, color='navy', label='Relevance vector')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Relevance Vector Regression')
#plt.legend()
plt.legend()
plt.show()

print("The length of relevance vetor is : {}".format(len(svr_rbf.support_vectors_)))
plt.figure()
lw=2
plt.scatter(X, y, color='darkorange', label='data')
plt.plot(X, y_svr, color='navy', lw=lw, label='RBF model')
plt.scatter(svr_rbf.support_vectors_, np.sin(svr_rbf.support_vectors_)/svr_rbf.support_vectors_,color='navy', label='Support vector')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Support Vector Regression')

plt.legend()
plt.show()

In [ ]:
X, y = make_friedman3()
print(X.shape)
print(y.shape)

In [ ]:
x=np.array([[3,5,2]]).reshape((-1,1))
y=np.array([[6,1,4]]).reshape((-1,1))
print(x.shape)
a = np.repeat(x,3,axis=1)
b=np.repeat(y.T,3,axis=0)
#print(a)
#print(x)
print(linear_kernel(x,y)) # np.dot(x,y.T)
#print(np.dot(x,y.T))
#Min= np.minimum(a,b)
#print(Min)
#print(np.dot(x.T,y))
#print(np.dot(x.T,y)*Min)
Min= np.minimum(a,b)
print(Min)
phi=1 + linear_kernel(x,y) + linear_kernel(x,y)*Min - (x+y.T)*np.square(Min)/2 + np.power(Min,3)
#np.dot((x+y.T),np.square(Min))
print(phi)


In [ ]:
print(__doc__)

import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
%matplotlib inline
# #############################################################################
# Generate sample data
X = np.sort(10 * np.random.rand(100, 1)-5, axis=0)
y = np.sin(X).ravel()

# #############################################################################
# Add noise to targets
y[::5] += 3 * (0.5 - np.random.rand(20))

# #############################################################################
# Fit regression model
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)
svr_poly = SVR(kernel='poly', C=1e3, degree=2)
y_rbf = svr_rbf.fit(X, y).predict(X)
y_lin = svr_lin.fit(X, y).predict(X)
y_poly = svr_poly.fit(X, y).predict(X)

# #############################################################################
# Look at the results
lw = 2
plt.scatter(X, y, color='darkorange', label='data')
plt.plot(X, y_rbf, color='navy', lw=lw, label='RBF model')
plt.plot(X, y_lin, color='c', lw=lw, label='Linear model')
plt.plot(X, y_poly, color='cornflowerblue', lw=lw, label='Polynomial model')
plt.xlabel('data')
plt.ylabel('target')
plt.title('Support Vector Regression')
plt.legend()
plt.show()

In [ ]:
np.random.normal(0,0.2)